# This notebook helps create AWS domain and creds

#### This cell sets default values 

In [ ]:
import sys
sys.path.append('..')
import mmAuthorization

import requests
import json, os, pprint
import base64
import getpass

### Following defines config values for later resuse

In [ ]:
# hostname of Viya4 server where ever you are using model manager from
host_name = "viya-server.company.com"
port = "PORT"

host_url="https://" + host_name 
destination_url = host_url + "/modelPublish/destinations/"
modelrepo_url = host_url + "/modelRepository/models/"
publishmodel_url = host_url + "/modelPublish/models"
domains_url = host_url + "/credentials/domains"

print(host_url)

# AWS secret keys of sunall_modeluser1 created thru IAM console. This particular one should have roles associated with 
# AmazonElasticContainerRegistry, AmazonEC2ContainerRegistryFullAccess* 
AWS_KEY_ID = ""
AWS_SECRET_KEY = ""

# admin id to connect to Viya4
admin_userId = 'whoami'
user_passwd = getpass.getpass()

# The user who can use the credentials.
USER_ID = "whoami"

domain_name = "fsbu_aws_domain_1"
domain_description = 'fsbu aws domain 1'

dest_name = "fsbu_aws_dest_1"
awsRegion = "us-east-1"
# following EKS is not required for publishing images. You can come with some arbitary name just to create  destination. You need working one only for model validation. 
k8sClusterName = "mm-docker-models-eks"


### Following gets Auth token

In [ ]:
mm_auth = mmAuthorization.mmAuthorization("myAuth")

admin_auth_token = mm_auth.get_auth_token(host_url, admin_userId, user_passwd)

credential_admin_headers = {
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

destination_harbor_headers = {
    "If-Match":"false",
    "Content-Type":"application/vnd.sas.models.publishing.destination.privatedocker+json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

print(admin_auth_token)

In [ ]:
##### create Domain

my_domain_url = domains_url + "/" + domain_name
domain_attrs = {
    "id":domain_name,
    "type":"base64",
    "description": domain_description
}

domain = requests.put(my_domain_url, 
                       data=json.dumps(domain_attrs), headers=credential_domain_headers, verify=False)

print(domain)
pprint.pprint(domain.json())

In [ ]:
### Create credential 
####
# Create credentials for the specified user.
my_credential_url = my_domain_url + "/users/" + USER_ID

key_id = AWS_KEY_ID
secret_access_Key = AWS_SECRET_KEY

encoded_key_id = str(base64.b64encode(key_id.encode("utf-8")), "utf-8")
encoded_access_key = str(base64.b64encode(secret_access_Key.encode("utf-8")), "utf-8")
print(encoded_access_key)

credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":USER_ID,
    "domainType":"base64",
    "properties":{"accessKeyId":encoded_key_id},
    "secrets":{"secretAccessKey":encoded_access_key}
}

credential = requests.put(my_credential_url, 
                       data=json.dumps(credential_attrs), headers=credential_user_headers, verify=False)

print(domain)
#pprint.pprint(domain.json())
print("The credentials have been created within the domain ", domain_name)

print(credential)
pprint.pprint(credential.json())

In [ ]:
# Creates a new destination, expecting a response code of 201.
destination_aws_post_headers = {
    "Content-Type":"application/vnd.sas.models.publishing.destination.aws+json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
} 


#        {"name": "repositoryName",
#         "value": "experian/scr-models"}

destination_attrs = {
    "name":dest_name,
    "destinationType":"aws",
    "properties":[
        {"name": "credDomainId",
        "value": domain_name},
        {"name": "region",
        "value": awsRegion},
        {"name": "kubernetesCluster",
        "value": k8sClusterName}
     ]
}

destination = requests.post(host_url + '/modelPublish/destinations', 
                       data=json.dumps(destination_attrs), headers=destination_aws_post_headers,verify=False)

print(destination)
pprint.pprint(destination.json())


## Get/List proccess from here on...

In [ ]:
def list_destinations(destination_url, auth_token): 
    headers = {
        mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + auth_token
    }

    print("List the destinations...")
    try:
        response = requests.get(destination_url, headers=headers,verify=False)
        jsondata = response.json()
        destinations = jsondata['items']
        if len(destinations) > 0:
            for destination in destinations:            
                print(destination["id"])
                print(destination["name"])
                print("===========")

    except:
        raise RuntimeError("ERROR: Could not get a destination list.")
        
list_destinations(destination_url, admin_auth_token)

In [ ]:
# get credentials list 

headersGet = {
    'Authorization': 'Bearer ' + admin_auth_token}

credentialURL = domains_url + "/" + domain_name + '/credentials'
print("cred URL:", credentialURL)

credentialGet = requests.get(credentialURL, headers=headersGet,verify=False)
print("http response code:", credentialGet)

for i, domain in enumerate(credentialGet.json()['items']):
    print(f"{i}. Domain ID: {domain['domainId']}, {domain['identityId']}, {domain['identityType']}")
 

## Delete proccess from here on...

In [ ]:
# Delete destination 

deletedURL = destination_url + dest_name
destination = requests.delete(deletedURL, headers=credential_admin_headers, verify=False)

print("Deleted destination:", deletedURL)
print(destination)
